In [12]:
import pandas as pd

In [13]:
data = pd.read_csv('powerconsumption.csv')

In [14]:
data['Datetime'] = pd.to_datetime(data['Datetime'])
data['Day'] = data['Datetime'].dt.day
data['Month'] = data['Datetime'].dt.month
data['Hour'] = data['Datetime'].dt.hour
data['Day of Week'] = data['Datetime'].dt.dayofweek + 1

data = data.drop(['Datetime'],axis=1)

In [15]:
from sklearn.model_selection import train_test_split

X = data.drop(['PowerConsumption_Zone1','PowerConsumption_Zone2','PowerConsumption_Zone3'], axis=1)
y = data['PowerConsumption_Zone1']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
def create_mlp_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [8]:
model = create_mlp_model(X_train.shape[1])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, batch_size=32, verbose=1)

Epoch 1/500
983/983 [==============================] - 3s 2ms/step - loss: 166367840.0000 - val_loss: 24990166.0000
Epoch 2/500
983/983 [==============================] - 2s 2ms/step - loss: 22827012.0000 - val_loss: 21258212.0000
Epoch 3/500
983/983 [==============================] - 2s 2ms/step - loss: 21035692.0000 - val_loss: 20150214.0000
Epoch 4/500
983/983 [==============================] - 2s 2ms/step - loss: 20499652.0000 - val_loss: 19833326.0000
Epoch 5/500
983/983 [==============================] - 2s 2ms/step - loss: 20128662.0000 - val_loss: 19503372.0000
Epoch 6/500
983/983 [==============================] - 2s 2ms/step - loss: 19920908.0000 - val_loss: 19440782.0000
Epoch 7/500
983/983 [==============================] - 2s 2ms/step - loss: 19497790.0000 - val_loss: 19152484.0000
Epoch 8/500
983/983 [==============================] - 2s 2ms/step - loss: 19186944.0000 - val_loss: 18572614.0000
Epoch 9/500
983/983 [==============================] - 2s 2ms/step - loss: 1867

In [9]:
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(model, X_train, Y_train, X_val, Y_val, X_test, Y_test):
    Y_train_pred = model.predict(X_train)
    Y_val_pred = model.predict(X_val)
    Y_test_pred = model.predict(X_test)

    print("Training MSE:", mean_squared_error(Y_train, Y_train_pred))
    print("Validation MSE:", mean_squared_error(Y_val, Y_val_pred))
    print("Test MSE:", mean_squared_error(Y_test, Y_test_pred))

    print("Training R-squared:", r2_score(Y_train, Y_train_pred))
    print("Validation R-squared:", r2_score(Y_val, Y_val_pred))
    print("Test R-squared:", r2_score(Y_test, Y_test_pred))

    print(Y_test[0:5])
    print(Y_test_pred[0:5])

evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test)

328/328 [==============================] - 0s 796us/step
Training MSE: 4008746.436809497
Validation MSE: 4152892.610869986
Test MSE: 4003412.6969335307
Training R-squared: 0.9212679374785825
Validation R-squared: 0.9184801172423318
Test R-squared: 0.9207156877841893
15506    32985.14532
51943    34737.64259
212      27894.68354
11210    42476.93617
16307    21341.14101
Name: PowerConsumption_Zone1, dtype: float64
[[32959.84 ]
 [37248.496]
 [30683.56 ]
 [43849.438]
 [23095.107]]


In [10]:
model.save('saved_model_directory')

INFO:tensorflow:Assets written to: saved_model_directory\assets


INFO:tensorflow:Assets written to: saved_model_directory\assets


In [11]:
import tensorflow as tf
import numpy as np

# Load your trained model
model = tf.keras.models.load_model('saved_model_directory')

# Extract weights and biases
weights = model.get_weights()

# Save to a C++ readable file
with open('mlp_weights.cpp', 'w') as f:
    for i, w in enumerate(weights):
        if len(w.shape) == 2:  # Dense layer weights (matrix)
            f.write(f"const float W{i}[][] = {{\n")
            for row in w:
                f.write("    {" + ", ".join(map(str, row)) + "},\n")
            f.write("};\n\n")
        elif len(w.shape) == 1:  # Dense layer biases (vector)
            f.write(f"const float B{i}[] = {{" + ", ".join(map(str, w)) + "};\n\n")

In [18]:
import pandas as pd

# Limit to 150 rows
X_test = X_test.iloc[:150]
y_test = y_test.iloc[:150]

# Define the C++ file name
filename = "test_data.cpp"

# Generate C++ array declarations
x_array_name = "x_test"
y_array_name = "y_test"

if X_test.empty:
    raise ValueError("X_test is empty")
if y_test.empty:
    raise ValueError("y_test is empty")

num_rows, num_cols = X_test.shape

# Write to the C++ file
with open(filename, mode='w') as file:
    # Write a comment and include guard
    file.write("// Test datasets exported from Python\n")
    
    # Write x_test array
    file.write(f"const int x_num_rows = {num_rows};\n")
    file.write(f"const int x_num_cols = {num_cols};\n")
    file.write(f"const float {x_array_name}[{num_rows}][{num_cols}] = {{\n")
    for row in X_test.values:  # Convert to NumPy array for iteration
        row_str = ", ".join(f"{value:.8f}" for value in row)  # Format numbers for precision
        file.write(f"    {{ {row_str} }},\n")
    file.write("};\n\n")
    
    # Write y_test array
    file.write(f"const int y_num_rows = {len(y_test)};\n")
    file.write(f"const float {y_array_name}[{len(y_test)}] = {{\n")
    y_str = ", ".join(f"{value}" for value in y_test)
    file.write(f"    {y_str}\n")
    file.write("};\n")

print(f"Data successfully exported to {filename}")


Data successfully exported to test_data.cpp
